# Parameter scan 

This code is used for making parameter scans. It is based on density estimation code version 3. See that file for explanations.
### Caution: 
The code below writes results to file and then reads them back in for analysis and comparison.
You cannot do a Run All before setting the file paths right!

In [ ]:
import pandas as pd
import json
import numpy as np
from scipy.stats import multivariate_normal, pearsonr
import copy
from importlib import reload
import dens_estimation as de
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

## Real dataset

In [ ]:
# real dataset

data = []
with open("/Users/philip/Documents/PhD/data/ArenaData/arena_fits/2015-07-05.json") as f:
    data = f.readlines()

json_lines = []

for line in data:
    jsline = json.loads(line)
    json_lines.append(jsline)

In [ ]:
frame = pd.DataFrame.from_dict(json_lines)

In [ ]:
frame.head()

In [ ]:
# rebuild dataframe
# make dataframe of dicts nested in 'value' column
value = pd.DataFrame(list(frame['value']))
del frame['value']

# make dataframe of dicts nested in 'trackeeHistory' column
trackee = pd.DataFrame(list(value['trackeeHistory']))
del value['trackeeHistory']

chi2PerDof = pd.DataFrame(list(trackee['chi2PerDof']))
chi2PerDof.columns = ['chi2PerDof']
probChi2 = pd.DataFrame(list(trackee['probChi2']))
probChi2.columns = ['probChi2']
nMeasurements = pd.DataFrame(list(trackee['nMeasurements']))
nMeasurements.columns = ['nMeasurements']
localMac = pd.DataFrame(list(trackee['localMac']))
localMac.columns = ['localMac']

In [ ]:
# make dataframe with a 'coordinates' column
averagecoordinate = pd.DataFrame(list(value['averagecoordinate']))
coordinates = pd.DataFrame(list(averagecoordinate['avg']))
averagecoordinate = averagecoordinate.join(coordinates)
error = pd.DataFrame(list(averagecoordinate['error']))
errorcoordinates = pd.DataFrame(list(error['coordinates']))
del errorcoordinates[2]
errorcoordinates.columns = ['x_error','y_error']

del averagecoordinate['avg']
del value['averagecoordinate']

# join dataframes
frame = frame.join(value.join(averagecoordinate))
frame = frame.join(chi2PerDof)
frame = frame.join(probChi2)
frame = frame.join(errorcoordinates)
frame = frame.join(localMac)
frame = frame.join(nMeasurements)
del frame['regionsNodesIds']
del frame['error']
del frame['type']

In [ ]:
frame = frame[frame['localMac'] == 0]

In [ ]:
frame = frame.sort_values(by='measurementTimestamp')

## Analysis code

In [ ]:
def selectWindow(k):
    start = min(df['measurementTimestamp']) + k * timestep
    stop = start + interval

    window = df[(df['measurementTimestamp'] >= start) & 
                       (df['measurementTimestamp'] < stop)]

    return window

In [ ]:
def createDataStructures(window):
    grids = np.zeros((len(set(window['sourceMac'])), height,width))

    # dictionary of histograms (with mac addresses as keys)
    histos = dict(zip(set(window['sourceMac']), grids))
    
    emptylist = [[] for i in range(len(set(window['sourceMac'])))]
    positions = dict(zip(set(window['sourceMac']), emptylist))
    emptylist = [[] for i in range(len(set(window['sourceMac'])))]
    x_errors = dict(zip(set(window['sourceMac']), emptylist))
    emptylist = [[] for i in range(len(set(window['sourceMac'])))]
    y_errors = dict(zip(set(window['sourceMac']), emptylist))
    
    history = dict(zip(set(window['sourceMac']), np.zeros(len(set(window['sourceMac'])))))
    
    return histos, positions, x_errors, y_errors, history

In [ ]:
def resetDataStructures(histos):
    
    histos_old = copy.deepcopy(histos)
    
    grids = np.zeros((len(histos), height,width))
    histos = dict(zip(histos.keys(), grids))
    
    emptylist = [[] for i in range(len(histos))]
    positions = dict(zip(histos.keys(), emptylist))
    emptylist = [[] for i in range(len(histos))]
    x_errors = dict(zip(histos.keys(), emptylist))
    emptylist = [[] for i in range(len(histos))]
    y_errors = dict(zip(histos.keys(), emptylist))
    
    return histos, histos_old, positions, x_errors, y_errors

In [ ]:
def updateDataStructures(window, histos, positions, x_errors, y_errors, history):
    for i in range(len(window)):
        if not window['sourceMac'].values[i] in positions:
            histos[window['sourceMac'].values[i]] = np.zeros((height,width))
            positions[window['sourceMac'].values[i]] = []
            x_errors[window['sourceMac'].values[i]] = []
            y_errors[window['sourceMac'].values[i]] = []
            history[window['sourceMac'].values[i]] = 0
            
        positions[window['sourceMac'].values[i]].append(window['coordinates'].values[i][:2])
        x_errors[window['sourceMac'].values[i]].append(window['x_error'].values[i])
        y_errors[window['sourceMac'].values[i]].append(window['y_error'].values[i])
        
    return histos, positions, x_errors, y_errors, history

In [ ]:
def createDensityEstimates(window, gridpoints, histos, positions, x_errors, y_errors):

    for mac in histos.keys():
        if len(positions[mac]) > 0:
            values = np.transpose(np.array(positions[mac]))
            uncertainties = np.array([x_errors[mac], y_errors[mac]])
            kernel = de.variable_kde(values, uncertainties)
            binvals = kernel(gridpoints)
            # reshape() stacks row-wise, so we use the Fortran-like index ordering
            estimate = np.reshape(binvals, (height,width), order='F')
            histos[mac] += estimate
            # here we don't renormalize the evaluation grid to unity
            '''
            if histos[mac].sum() > 0:
                histos[mac] /= histos[mac].sum()'''
    return histos

In [ ]:
def memorizeNonUpdatedEstimates(histos, histos_old, positions, history, memory):
    for mac in histos.keys():
        if len(positions[mac]) == 0:
            if history[mac] < memory:
                histos[mac] += histos_old[mac]
                history[mac] += 1
            else:
                history[mac] = 0
    return histos

In [ ]:
def sumHistograms(histos):
    # total density histogram per period
    total_dens_histo = np.zeros((height, width))
    
    for mac in histos.keys():
        total_dens_histo += histos[mac]
                
    return total_dens_histo

In [ ]:
def runDataAnalysis():
    
    for k in range(periods):
        window = selectWindow(k)
        if k < 1:
            histos, positions, x_errors, y_errors, history = createDataStructures(window)
            histos, positions, x_errors, y_errors, history =\
            updateDataStructures(window, histos, positions, x_errors, y_errors, history)
            histos = createDensityEstimates(window, gridpoints, histos, positions, x_errors, y_errors)
        else:
            histos, histos_old, positions, x_errors, y_errors = resetDataStructures(histos)
            histos, positions, x_errors, y_errors, history =\
            updateDataStructures(window, histos, positions, x_errors, y_errors, history)
            histos = createDensityEstimates(window, gridpoints, histos, positions, x_errors, y_errors)
            histos = memorizeNonUpdatedEstimates(histos, histos_old, positions, history, memory)

        total_dens_histo = sumHistograms(histos)
        if k == (periods - 1):
            np.savetxt('output/scan_histo_%d_%d_%d.csv' % (memory, interval, bins**2), total_dens_histo, delimiter=',')
            print('Output written to scan_histo_%d_%d_%d.csv' % (memory, interval, bins**2))

## Parameter scan

The code below sets up things to scan 3 parameters: 
    - memory
    - time window interval
    - number of evaluation grid points
    
We have focused on a 15 x 15 meter square area from north east coordinates (39,-39) 
to south west coordintes (54,-24).
We divide this square by {2,3,4,...} bins to generate {4,9,16,...} evaluation grid points.
We shift the resulting evaluation grid points to position them in the center of imaginary 'cells' or bins.

The function runDataAnalysis is adapted to write only the last time window estimate to file.
Only this last time window estimate coincides in time with the video moment.
The start point for each run is the video moment minus the memory times the time window.
We select a smaller dataframe 'df' which is subset of the dataframe 'frame', starting at the start point.
The function selectWindow works on the smaller dataframe 'df'.

In [ ]:
reload(de)

memory_parameter_set = np.arange(10)
interval_parameter_set = np.arange(10000,110000,10000) # (20000,140000,20000)
cellsize_parameter_set = np.array([2,3,4])

# 05:32:04 +2:00 UTC
timepoint = 1436067124000
lattice = 15

for bins in cellsize_parameter_set:
    cellsize = lattice / bins
    height = width = bins
    X, Y = np.mgrid[39:54:cellsize,-39:-24:cellsize]
    X = X + cellsize/2
    Y = Y + cellsize/2
    # note: ravel() concatenates columns
    gridpoints = np.vstack([X.ravel(), Y.ravel()])
    for m in memory_parameter_set:
        memory = m
        for t_int in interval_parameter_set:
            periods = m + 1
            timestep = t_int # 30000
            interval = t_int
            start_time = timepoint - periods * interval
            df = frame[frame['measurementTimestamp'] > start_time]
            runDataAnalysis()

## Compare results

### Make sure the file paths are correct! 

We first read in the video count data, and bin it to make it similar to the wi-fi density estimates.
The video count data has switched x- and y-axes, and then the y-axis need to be mirrored in the x-axis to
make the data correspond to the wi-fi coordinate system.

We assume the parameter arrays are still in memory.

In [ ]:
heads = np.loadtxt('/Users/philip/Documents/PhD/data-analysis/video/Sensation2015/\
movie1-01m12s/headcount-locations-manually.csv',delimiter=',')

# first swap columns, then mirror y-coordinates in x-axis 
# to be consistent with wi-fi coordinates
heads[:,[0, 1]] = heads[:,[1, 0]]
heads[:,1] = -heads[:,1]

corr_coeff = np.zeros((len(memory_parameter_set),len(interval_parameter_set),\
                       len(cellsize_parameter_set)))
RMSE = np.zeros((len(memory_parameter_set),len(interval_parameter_set),\
                       len(cellsize_parameter_set)))

for k in range(len(cellsize_parameter_set)):
    bins = cellsize_parameter_set[k]
    cellsize = lattice / bins
    # bin head counts in two-dimensional array
    video_estimate = np.zeros((bins, bins))
    for b in range(len(heads)):
        if heads[b][0] > 39 and heads[b][0] < 54 and heads[b][1] > -39 and heads[b][1] < -24:
            x = int((heads[b][0] - 39) / cellsize)
            y = int((heads[b][1] - (-39)) / cellsize)
            video_estimate[y][x] += 1
    #### now we have video_estimate in the same format as the wifi_estimate to be loaded
    for i in range(len(memory_parameter_set)):
        m = memory_parameter_set[i]
        for j in range(len(interval_parameter_set)):
            t_int = interval_parameter_set[j]
            wifi_estimate = np.loadtxt('/Users/philip/PycharmProjects/DensityEstimation/'
            'output/scan_histo_%d_%d_%d.csv' % (m, t_int, bins**2), delimiter=',')
            # now look at the correlation coefficient between the two distributions
            X = video_estimate.ravel()
            Y = wifi_estimate.ravel()
            corr_coeff[i,j,k] = pearsonr(X, Y)[0]
            RMSE[i,j,k] = abs(X.sum() - Y.sum())

In [ ]:
corr_coeff.shape

In [ ]:
corr_coeff.max()

## Plotting

In [ ]:
from matplotlib import cm

fig = plt.figure(figsize=(16,12))
for i in range(corr_coeff.shape[2]):
    ax = fig.add_subplot(2,2,i+1, projection='3d')
    a = interval_parameter_set/1000
    b = memory_parameter_set
    x_data, y_data = np.meshgrid(a,b)
    zs = corr_coeff[:,:,i].ravel()
    z_data = zs.reshape(x_data.shape)
    ax.plot_surface(x_data, y_data, z_data, rstride=1, cstride=1, linewidth=1, antialiased=False, alpha=0.5)
    plt.xlabel('Time window [s]')
    plt.ylabel('Memory')
    ax.set_zlabel('pearson r')
    plt.title('Lattice %dx%d' % (cellsize_parameter_set[i],cellsize_parameter_set[i]))
#plt.savefig('/Users/philip/Documents/PhD/data-analysis/video/Sensation2015/movie1-01m12s/param-scan-03-3dsurf.png')
plt.show()

In [ ]:
fig = plt.figure(figsize=(16,12))
for i in range(RMSE.shape[2]):
    ax = fig.add_subplot(2,2,i+1, projection='3d')
    a = interval_parameter_set/1000
    b = memory_parameter_set
    x_data, y_data = np.meshgrid(a,b)
    zs = RMSE[:,:,i].ravel()
    z_data = zs.reshape(x_data.shape)
    ax.plot_surface(x_data, y_data, z_data, rstride=1, cstride=1, linewidth=1, antialiased=False, alpha=0.5)
    plt.xlabel('Time window [s]')
    plt.ylabel('Memory')
    ax.set_zlabel('RMSE')
    plt.title('Lattice %dx%d' % (cellsize_parameter_set[i],cellsize_parameter_set[i]))
#plt.savefig('/Users/philip/Documents/PhD/data-analysis/video/Sensation2015/movie1-01m12s/param-scan-03-rmse.png')
plt.show()

In [ ]:
for i in range(3):
    print('Lattice size:', cellsize_parameter_set[i]**2, '; max pearson:', corr_coeff[:,:,i].max())

In [ ]:
max_index = np.unravel_index(np.argmax(corr_coeff), corr_coeff.shape)
print('Optimal parameters:','\n',
      'Lattice size:', '%dx%d' % (cellsize_parameter_set[max_index[2]],cellsize_parameter_set[max_index[2]]),'\n',
      'Memory:', memory_parameter_set[max_index[0]],'\n',
      'Time window (s):', interval_parameter_set[max_index[1]]/1000)

In [ ]:
corr_coeff[8,9,2]

In [ ]:
corr_coeff[8,9,2]==corr_coeff[:,:,2].max()

In [ ]:
# create scatter plot of estimates with max correlation coefficient
# re-create video estimate histo
bins = cellsize_parameter_set[2] # max_index[2]
cellsize = lattice / bins
# bin head counts in two-dimensional array
video_estimate = np.zeros((bins, bins))
for b in range(len(heads)):
    if heads[b][0] > 39 and heads[b][0] < 54 and heads[b][1] > -39 and heads[b][1] < -24:
        x = int((heads[b][0] - 39) / cellsize)
        y = int((heads[b][1] - (-39)) / cellsize)
        video_estimate[y][x] += 1
# re-create wi-fi estimate histo
m = memory_parameter_set[max_index[0]] # max_index[0]
t_int = interval_parameter_set[max_index[1]] # max_index[1]
wifi_estimate = np.loadtxt('/Users/philip/PycharmProjects/DensityEstimation/'
'output/scan_histo_%d_%d_%d.csv' % (m, t_int, bins**2), delimiter=',')
# now look at the correlation coefficient between the two distributions
X = video_estimate.ravel()
Y = wifi_estimate.ravel()

In [ ]:
X.max()

In [ ]:
fig = plt.figure(figsize=(15,4)) # 

ax = fig.add_subplot(1,1,1)
plt.plot(X,Y,'ro')
plt.xlim([X.min()-1,X.max()+1])
plt.ylim([Y.min()-50,Y.max() + 50])
plt.xlabel('Video estimates')
plt.ylabel('Wi-Fi estimates')
plt.title('2x2 lattice (r = ...)')
#plt.savefig('/Users/philip/Documents/PhD/data-analysis/video/Sensation2015/movie1-01m12s/scatter-plot-01a.png')
plt.show()